# AI Comic Book Generator - Colab Launcher

This notebook runs the AI Comic Generator **completely FREE** using local Ollama models.
**No API keys needed!** Everything runs on Colab's free GPU.

In [ ]:
# @title 1. Initial Setup (Run Once)
# @markdown Installs Ollama, zstd, and clones the repository.
REPO_URL = "https://github.com/yogesh-dixit-dev/comic-generator.git" # @param {type:"string"}
BRANCH = "master" # @param {type:"string"}

import os
import sys

# Install zstd (required by Ollama install script)
print("📦 Installing dependencies (zstd)...")
os.system("apt-get update -qq && apt-get install -y -qq zstd")

# Install Ollama
print("📦 Installing Ollama...")
os.system("curl -fsSL https://ollama.com/install.sh | sh")

# Start Ollama server in background
print("🚀 Starting Ollama server...")
os.system("nohup ollama serve > ollama.log 2>&1 &")

# Pull the models
import time
time.sleep(5) # Wait for server
print("⬇️ Downloading Reasoning Model (Llama 3.1 8B - this may take 5-10 mins)...")
os.system("ollama pull llama3.1")

print("⬇️ Downloading Fast Model (Llama 3.2 3B)...")
os.system("ollama pull llama3.2")

# Clone or update repo
if not os.path.exists("comic-gen"):
    os.system(f"git clone -b {BRANCH} {REPO_URL} comic-gen")

os.chdir("/content/comic-gen")
os.system("pip install -q -r requirements.txt")

print("\n✅ Core Installation Complete!")

In [ ]:
# @title 2. Update Code (Run after every fix!)
# @markdown Pulls the latest changes from GitHub WITHOUT re-installing everything.
import os
if os.path.exists('/content/comic-gen'):
    os.chdir('/content/comic-gen')
    print("🔄 Pulling latest code...")
    os.system("git fetch --all")
    os.system("git reset --hard origin/master")
    print("\n✅ Code Updated to Latest Version!")

In [ ]:
# @title 2b. Pull Missing Models (Quick Fix)
# @markdown Run this if you get a "model not found" error.
import os
os.system("ollama pull llama3.1")
os.system("ollama pull llama3.2")
print("\n✅ Models Ready!")

In [ ]:
# @title 3. Configuration (Optional - Hugging Face Storage)
import os
from google.colab import userdata

try:
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    print("✅ Loaded HF token from Colab Secrets")
except:
    print("⚠️ No HF_TOKEN in Colab Secrets. Output will stay local.")
    pass

In [ ]:
# @title 4. Progress Tracking Helper
import subprocess
import re
from tqdm.notebook import tqdm

def run_pipeline_with_progress(command, phase_name):
    print(f"🚀 Starting {phase_name} Phase...")
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    
    pbar = tqdm(total=100, desc=phase_name, unit="%")
    last_progress = 0
    
    for line in iter(process.stdout.readline, ''):
        # Print regular logs but skip noisy progress markers for cleaner UI
        if "[PROGRESS]" in line:
            match = re.search(r"\[PROGRESS\] (\d+)%", line)
            if match:
                progress = int(match.group(1))
                pbar.update(progress - last_progress)
                last_progress = progress
        else:
            print(line, end='')
            
    pbar.n = 100
    pbar.refresh()
    process.wait()
    if process.returncode == 0:
        print(f"\n✅ {phase_name} complete!")
    else:
        print(f"\n❌ {phase_name} failed with code {process.returncode}")

In [ ]:
# @title 5. Phase 1: Knowledge & Visual Planning
# @markdown Runs narrative scripting, character design, and panel planning.
INPUT_FILE = "story.txt" # @param {type:"string"}
OUTPUT_DIR = "output" # @param {type:"string"}
USE_HF_STORAGE = False # @param {type:"boolean"}
HF_REPO_ID = "your-username/comic-dataset" # @param {type:"string"}

import os
os.chdir('/content/comic-gen')
storage_flag = f"--storage hf --hf_repo '{HF_REPO_ID}'" if USE_HF_STORAGE and os.environ.get('HF_TOKEN') else "--storage local"

cmd = f"python src/main.py --input '{INPUT_FILE}' --output '{OUTPUT_DIR}' --phase plan --colab {storage_flag}"
run_pipeline_with_progress(cmd, "Planning")

In [ ]:
# @title 6. Phase 2: Isolated Image Generation
# @markdown Loads the vision plans and draws high-quality images with full GPU dedicated to SDXL.
INPUT_FILE = "story.txt" # @param {type:"string"}
OUTPUT_DIR = "output" # @param {type:"string"}
USE_HF_STORAGE = False # @param {type:"boolean"}
HF_REPO_ID = "your-username/comic-dataset" # @param {type:"string"}

import os
os.chdir('/content/comic-gen')
storage_flag = f"--storage hf --hf_repo '{HF_REPO_ID}'" if USE_HF_STORAGE and os.environ.get('HF_TOKEN') else "--storage local"

cmd = f"python src/main.py --input '{INPUT_FILE}' --output '{OUTPUT_DIR}' --phase draw --colab {storage_flag}"
run_pipeline_with_progress(cmd, "Drawing")

In [ ]:
# @title 7. Download Output
from google.colab import files
import os

if os.path.exists('output'):
    os.system("zip -r comic_output.zip output")
    files.download('comic_output.zip')
    print("✅ Download started!")
else:
    print("⚠️ No output folder found. Did the pipeline run successfully?")